In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os,time
import DICOMPUTE as D

In [3]:
bucket = D.GC.get_bucket('ct-lymph-nodes')

Search completed after 1.3196616172790527 seconds.


In [4]:
print(D.GC.get_bucket_size(bucket))

Getting size completed after 14.706776142120361 seconds.
57794974010


In [5]:
patients = sorted(D.GC.lsdir(bucket, 'dicom/'))

Listing directory completed after 2.478931427001953 seconds.


In [6]:
len(patients), patients[0]

(176, 'dicom/61.7.102432584850582937613178565576727689438/')

In [7]:
studies = sorted(D.GC.lsdir(bucket, patients[0]))

Listing directory completed after 1.0038797855377197 seconds.


In [8]:
len(studies), studies[0]

(1,
 'dicom/61.7.102432584850582937613178565576727689438/61.7.236153238415899774813399005901015124149/')

In [9]:
images = D.GC.download(bucket, studies[0], outdir='/mnt/data/ct-lymph-nodes/')

Download completed after 1.044142484664917 seconds.


In [10]:
images[0]

'/mnt/data/ct-lymph-nodes/dicom/61.7.102432584850582937613178565576727689438/61.7.236153238415899774813399005901015124149/61.7.100084279359689996916137347486987360356.dcm'

In [11]:
#
# dcm2niix
#
dcm_dir = os.path.dirname(images[0])
nii_dir = dcm_dir.replace('dicom', 'nii')
nii_file = os.path.join(os.path.dirname(nii_dir), os.path.basename(dcm_dir))

nii_path = D.Processing.dcm2nii(dcm_dir, nii_file)

Conversion completed after  0.00019693374633789062 seconds.


In [12]:
#
# 3D SIFT
#
feature_file = D.Processing.sift3d(nii_path)

SIFT Calculation completed after 3.337860107421875e-05 seconds.


In [19]:
#
# pydra test
#
import pydra

In [20]:
@pydra.mark.task
def toNiiAndSift(which):
    dcm_dir = os.path.dirname(which)
    nii_dir = dcm_dir.replace('dicom', 'nii')
    nii_file = os.path.join(os.path.dirname(nii_dir), os.path.basename(dcm_dir))

    nii_path = D.Processing.dcm2nii(dcm_dir, nii_file)
    
    feature_file = D.Processing.sift3d(nii_path)
    
    return feature_file

In [21]:
task1_ex1 = toNiiAndSift(which=images[0:8]).split('which')
t0 = time.time()
task1_ex1()
print(f'total time: {time.time() - t0}')
task1_ex1.result()

RuntimeError: This event loop is already running